In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## CNN
http://cs231n.github.io/

## RNN
http://web.stanford.edu/class/cs224n/

https://www.youtube.com/watch?v=vT1JzLTH4G4&list=PL3FW7Lu3i5JvHM8ljYj-zLfQRF3EO8sYv

## Siraj Raval
https://www.youtube.com/channel/UCWN3xxRkmTPmbKwht9FuE5A/videos

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

batch_size = 64
test_batch_size = 10000

learning_rate = 0.01

epochs = 4

log_interval = 4

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
model = Net().to(device)

In [6]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.5)

In [7]:
def train(epoch):
    model.train() #sets module to training mode
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data) #forward pass
        loss = F.nll_loss(output, target) #negative log likelihood
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [8]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad(): #Disabling gradient calc is useful for inference, reduces memory consumption
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [9]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.345145
Train Epoch: 1 [256/60000 (0%)]	Loss: 2.315320
Train Epoch: 1 [512/60000 (1%)]	Loss: 2.335392
Train Epoch: 1 [768/60000 (1%)]	Loss: 2.279897
Train Epoch: 1 [1024/60000 (2%)]	Loss: 2.277301
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.306340
Train Epoch: 1 [1536/60000 (3%)]	Loss: 2.230925
Train Epoch: 1 [1792/60000 (3%)]	Loss: 2.246297
Train Epoch: 1 [2048/60000 (3%)]	Loss: 2.253626
Train Epoch: 1 [2304/60000 (4%)]	Loss: 2.190436
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.258265
Train Epoch: 1 [2816/60000 (5%)]	Loss: 2.214667
Train Epoch: 1 [3072/60000 (5%)]	Loss: 2.174774
Train Epoch: 1 [3328/60000 (6%)]	Loss: 2.225538
Train Epoch: 1 [3584/60000 (6%)]	Loss: 2.153151
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.138760
Train Epoch: 1 [4096/60000 (7%)]	Loss: 2.136610
Train Epoch: 1 [4352/60000 (7%)]	Loss: 1.973643
Train Epoch: 1 [4608/60000 (8%)]	Loss: 2.090150
Train Epoch: 1 [4864/60000 (8%)]	Loss: 2.071325
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.98282

Train Epoch: 1 [43008/60000 (72%)]	Loss: 0.553810
Train Epoch: 1 [43264/60000 (72%)]	Loss: 0.424970
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.517506
Train Epoch: 1 [43776/60000 (73%)]	Loss: 0.403237
Train Epoch: 1 [44032/60000 (73%)]	Loss: 0.692701
Train Epoch: 1 [44288/60000 (74%)]	Loss: 0.543227
Train Epoch: 1 [44544/60000 (74%)]	Loss: 0.429091
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.781026
Train Epoch: 1 [45056/60000 (75%)]	Loss: 0.432156
Train Epoch: 1 [45312/60000 (75%)]	Loss: 0.444959
Train Epoch: 1 [45568/60000 (76%)]	Loss: 0.412020
Train Epoch: 1 [45824/60000 (76%)]	Loss: 0.388771
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.525177
Train Epoch: 1 [46336/60000 (77%)]	Loss: 0.729021
Train Epoch: 1 [46592/60000 (78%)]	Loss: 0.518381
Train Epoch: 1 [46848/60000 (78%)]	Loss: 0.415182
Train Epoch: 1 [47104/60000 (78%)]	Loss: 0.515184
Train Epoch: 1 [47360/60000 (79%)]	Loss: 0.629662
Train Epoch: 1 [47616/60000 (79%)]	Loss: 0.802126
Train Epoch: 1 [47872/60000 (80%)]	Loss: 0.520620


Train Epoch: 2 [25344/60000 (42%)]	Loss: 0.329817
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.239187
Train Epoch: 2 [25856/60000 (43%)]	Loss: 0.431563
Train Epoch: 2 [26112/60000 (43%)]	Loss: 0.240244
Train Epoch: 2 [26368/60000 (44%)]	Loss: 0.347863
Train Epoch: 2 [26624/60000 (44%)]	Loss: 0.560405
Train Epoch: 2 [26880/60000 (45%)]	Loss: 0.398489
Train Epoch: 2 [27136/60000 (45%)]	Loss: 0.287036
Train Epoch: 2 [27392/60000 (46%)]	Loss: 0.384715
Train Epoch: 2 [27648/60000 (46%)]	Loss: 0.367439
Train Epoch: 2 [27904/60000 (46%)]	Loss: 0.442678
Train Epoch: 2 [28160/60000 (47%)]	Loss: 0.421360
Train Epoch: 2 [28416/60000 (47%)]	Loss: 0.335848
Train Epoch: 2 [28672/60000 (48%)]	Loss: 0.317058
Train Epoch: 2 [28928/60000 (48%)]	Loss: 0.387505
Train Epoch: 2 [29184/60000 (49%)]	Loss: 0.433849
Train Epoch: 2 [29440/60000 (49%)]	Loss: 0.325593
Train Epoch: 2 [29696/60000 (49%)]	Loss: 0.360250
Train Epoch: 2 [29952/60000 (50%)]	Loss: 0.405030
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.318285


Train Epoch: 3 [7680/60000 (13%)]	Loss: 0.404574
Train Epoch: 3 [7936/60000 (13%)]	Loss: 0.270533
Train Epoch: 3 [8192/60000 (14%)]	Loss: 0.324477
Train Epoch: 3 [8448/60000 (14%)]	Loss: 0.419163
Train Epoch: 3 [8704/60000 (14%)]	Loss: 0.271539
Train Epoch: 3 [8960/60000 (15%)]	Loss: 0.294268
Train Epoch: 3 [9216/60000 (15%)]	Loss: 0.312760
Train Epoch: 3 [9472/60000 (16%)]	Loss: 0.400273
Train Epoch: 3 [9728/60000 (16%)]	Loss: 0.375907
Train Epoch: 3 [9984/60000 (17%)]	Loss: 0.287564
Train Epoch: 3 [10240/60000 (17%)]	Loss: 0.278688
Train Epoch: 3 [10496/60000 (17%)]	Loss: 0.263258
Train Epoch: 3 [10752/60000 (18%)]	Loss: 0.341608
Train Epoch: 3 [11008/60000 (18%)]	Loss: 0.270828
Train Epoch: 3 [11264/60000 (19%)]	Loss: 0.360919
Train Epoch: 3 [11520/60000 (19%)]	Loss: 0.380548
Train Epoch: 3 [11776/60000 (20%)]	Loss: 0.400804
Train Epoch: 3 [12032/60000 (20%)]	Loss: 0.279454
Train Epoch: 3 [12288/60000 (20%)]	Loss: 0.318161
Train Epoch: 3 [12544/60000 (21%)]	Loss: 0.154012
Train Epoc

Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.179357
Train Epoch: 3 [50176/60000 (84%)]	Loss: 0.177340
Train Epoch: 3 [50432/60000 (84%)]	Loss: 0.418398
Train Epoch: 3 [50688/60000 (84%)]	Loss: 0.345584
Train Epoch: 3 [50944/60000 (85%)]	Loss: 0.136351
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.378034
Train Epoch: 3 [51456/60000 (86%)]	Loss: 0.277264
Train Epoch: 3 [51712/60000 (86%)]	Loss: 0.458807
Train Epoch: 3 [51968/60000 (87%)]	Loss: 0.110936
Train Epoch: 3 [52224/60000 (87%)]	Loss: 0.350017
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.092744
Train Epoch: 3 [52736/60000 (88%)]	Loss: 0.287494
Train Epoch: 3 [52992/60000 (88%)]	Loss: 0.306388
Train Epoch: 3 [53248/60000 (89%)]	Loss: 0.340530
Train Epoch: 3 [53504/60000 (89%)]	Loss: 0.268156
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.179307
Train Epoch: 3 [54016/60000 (90%)]	Loss: 0.090653
Train Epoch: 3 [54272/60000 (90%)]	Loss: 0.642345
Train Epoch: 3 [54528/60000 (91%)]	Loss: 0.344527
Train Epoch: 3 [54784/60000 (91%)]	Loss: 0.326037


Train Epoch: 4 [32256/60000 (54%)]	Loss: 0.242591
Train Epoch: 4 [32512/60000 (54%)]	Loss: 0.300111
Train Epoch: 4 [32768/60000 (55%)]	Loss: 0.323013
Train Epoch: 4 [33024/60000 (55%)]	Loss: 0.170989
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.155032
Train Epoch: 4 [33536/60000 (56%)]	Loss: 0.205122
Train Epoch: 4 [33792/60000 (56%)]	Loss: 0.200034
Train Epoch: 4 [34048/60000 (57%)]	Loss: 0.355812
Train Epoch: 4 [34304/60000 (57%)]	Loss: 0.261224
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.177404
Train Epoch: 4 [34816/60000 (58%)]	Loss: 0.264529
Train Epoch: 4 [35072/60000 (58%)]	Loss: 0.097700
Train Epoch: 4 [35328/60000 (59%)]	Loss: 0.320049
Train Epoch: 4 [35584/60000 (59%)]	Loss: 0.152160
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.172877
Train Epoch: 4 [36096/60000 (60%)]	Loss: 0.276918
Train Epoch: 4 [36352/60000 (61%)]	Loss: 0.221261
Train Epoch: 4 [36608/60000 (61%)]	Loss: 0.334340
Train Epoch: 4 [36864/60000 (61%)]	Loss: 0.290109
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.354197


In [10]:
? torch.no_grad